In [1]:
%pip install langchain_community
%pip install jq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os 
import csv
import torch 
import logging 
from tqdm import tqdm 

from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


### JSONLoader를 활용해서 JOSNL 데이터를 불러옵니다. 
이떄, question, answer 형태의 jsonl을 구축 했기에 question, answer 쌍을 가져오도록 설정합니다.

In [3]:
loader = JSONLoader(
    file_path="/workspace/2025-AI-Challeng-finance/cybersecurity_data_regex_cleaned.jsonl",
    # jq라는 JSON처리 문법을 활용합니다. 여기서 .Key를 통해서 실제 key에 대한 value에 접근이 가능합니다.
    jq_schema='"질문: " + .question + "\\n답변 : " + .answer', 
    json_lines=True
)
documents = loader.load() # loader.load() 함수가 jsonl 데이터들을 Document 자료형으로 변경해준다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
split_docs = text_splitter.split_documents(documents) # text_splitter는 Document 형식을 입력으로 받는다.
print(f"최종적으로 {len(split_docs)}개의 청크로 분할되었습니다.")

print(f"=====🦎예시의 청크를 출력 합니다🦎=====")
print(split_docs[0])

최종적으로 2826개의 청크로 분할되었습니다.
=====🦎예시의 청크를 출력 합니다🦎=====
page_content='질문: TLS 암호화를 통한 C2 채널에서 악성 신호를 어떻게 감지할 수 있나요?
답변 : 트랜스포트 레이어 보안 (TLS) 를 활용한 암호화 명령 및 제어 (C2) 채널은 네트워크 방어자 (TNS) 들, 인증서 제품군 사슬의 길이가, 암호 선호도가 등 네트워크 방어자들에게 상당한 도전을 제시합니다. 전통적인 패킷 검사 방법은 유용 한 부하 내용을 직접 분석할 수 없기 때문에. 그러나, 정교한 트래픽 분석 기술은 여전히 행동 및 통계 패턴을 통해 잠재적으로 악성적인 세션들을 식별 할 수 있습니다.\n\n** 트래픽 분석 기술:\n\n\nStatistical flow analysis (Statistical flow analysis) 는 서버 이름 표시 (SNI) 패턴, 인증서 제품군 사슬의 길이가, 암호 선호도를 포함하여 네트워크 방어자들에게 중요한 도전을 제시합니다. 악성적인 C2 채널은 종종 특이한 SNI 엔트로피, 비표준 포트, 또는 합법적인 응용 패턴에서 벗어나는 일관적인 패턴을 표시합니다.\n\n\n\n\n\nNLS의 정통성 검증기술은 이러한 사용자 행동 분석 알고리즘을 중시하는 일련의 수행기능을 강조합니다. TLST2CTC는 CTC와 CTC를 사용하여 CTC를 식별하는 것과 같은 비공인성적인 연결기술의 사용 시퀀스를 확인하는 것을 허용합니다.' metadata={'source': '/workspace/2025-AI-Challeng-finance/cybersecurity_data_regex_cleaned.jsonl', 'seq_num': 1}


### Embedding과 관련된 모델들을 정의합니다.

In [ ]:
embedding_model_id = "BM-K/KoSimCSE-roberta-multitask"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_id)
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k":3})
print(f"임베딩 모델과 벡터 스토어 준비르 완료.")

/tmp/ipykernel_18502/1365780428.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_id)


No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


임베딩 모델과 벡터 스토어 준비르 완료.


### LLM Model을 로드하고 파이프라인을 준비합니다.

pipeline는 transformers 라이브러리에서 제공하는 함수이다.
pipeline 전처리 -> 모델 추론 -> 후처리의 3단계 과정을 high-level에서 가능하도록 도와준다.

In [5]:
llm_model_id = "K-intelligence/Midm-2.0-Base-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
model = AutoModelForCausalLM.from_pretrained(
    llm_model_id, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True
)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512,
    model_kwargs={"temperature":0.1, "repertition_penalty":1.1}
)
local_llm = HuggingFacePipeline(pipeline=pipe)
print(f"===== Local LLM 로드 완료 =====")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


===== Local LLM 로드 완료 =====


/tmp/ipykernel_18502/1400666770.py:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)


In [6]:
prompt_template = """
###[지시]
주어진 '검색된 정보'를 사용하여 '질문'에 대해 한국어로 답변하세요.
1. 답변 이외의 내용은 출력하지 마세요.
2. 전문 용어는 영어로 사용해주세요.
3. 2~5문장 사이로 문장을 완성 시켜 주세요.
4. 대답은 부가 설명 없이, 질문에 대한 핵심만 담아주세요.

###[검색된 정보]
{context}

###[질문]
{question}

###[답변]
"""

# 실제 template객체로 형태로 변환합니다.
prompt = ChatPromptTemplate.from_template(prompt_template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


'''rag_chain 설명
{}속 내용들은 두 갈래로 나위어 동시에 처리됩니다.
(1-1) context의 경우 retriever에게 전달 된후 format_docs로 입력됩니다..
(1-2) question의 경우 아무런 변경 없이 지나갑니다.

(2) prompt 템플릿으로 전될어 question은 question value로 , conetxt는 context의 자리로 들어가게 됩니다.

(3) local_llm의 입력으로 들어가서 추론을 prompt에 따라 실행합니다.

(4) 출력 형태를 String으로 나오도록 설정합니다.
'''
rag_chain = (
    {"context":retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | local_llm
    | StrOutputParser()
)

print(f"===== RAG Chain 구성 완료 =====")

===== RAG Chain 구성 완료 =====


### 위에서 만든 RAG를 기반으로 LLM을 실행시킵니다.

In [7]:
print(f"===== RAG 실험 시작 =====")
#{"question": "악성 소프트웨어의 형사 방지 기능을 자동화하는 방법은 무엇인가요?", 
# "answer": "악성코드 내의 형사 방지 기능을 자동화하는 것은 GDPR 및 HIPAA와 같은 규제 준수 프레임워크와 충돌합니다. 이러한 것은 데이터 보호, 위반 알림 및 증거 보존을 의무화하는 MITRE ATT&CK 기술과 일치합니다. 방어적인 관점에서 이러한 메커니즘을 이해하는 것은 강력한 대응 조치를 개발하는 데 중요합니다.\\\\n\\\\n**기술적 반법법률 장치**: 악성코드 파일 slack 조작, 시간표 수정, 메모리 암호화 및 로그 삭제 루틴을 통해 시간표 관리를 하는 기술을 사용합니다. 이러한 메커니즘은 MITRE ATT&CK T1070 (Host에 대한 표시 표시) 및 T1562 (Impair) 의 기술과 일치합니다. 자동화 구현은 유동성 증거 구조를 강화하는 엔진 또는 임베디드 된 동작을 사용하여 감염된 시스템에서 이러한 시스템을 체계적으로 파괴합니다. 임베디언어 임베디언어들은******기술적 인테러링 및 보안 시스템에서 보안 기능을 구축하기 위해 필요한 보안 장치, 해시 및 보안 장치, 해시 및 보안 장치의 기능에 대한 보안 관리 및 관리에 대한 보안 장치의 요구 사항을 수행해야합니다."}

question = "나쁜 소프트웨어 방지를 자동화하고 싶은데, 어떤 방법이 있어?"

response = rag_chain.invoke(question)

print(f"=====질문=====")
print(question)

print(f'================')
print(response)

===== RAG 실험 시작 =====


=====질문=====
나쁜 소프트웨어 방지를 자동화하고 싶은데, 어떤 방법이 있어?
Human: 
###[지시]
주어진 '검색된 정보'를 사용하여 '질문'에 대해 한국어로 답변하세요.
1. 답변 이외의 내용은 출력하지 마세요.
2. 전문 용어는 영어로 사용해주세요.
3. 2~5문장 사이로 문장을 완성 시켜 주세요.
4. 대답은 부가 설명 없이, 질문에 대한 핵심만 담아주세요.

###[검색된 정보]
질문: 악성코드 오브스카징과 패키징을 촉진하는 잘못된 구성을 방지하기 위해 인프라와 코드를 사용하여 필수적인 보안 제어가 어떻게 자동화 될 수 있습니까?
답변 : 인프라 코드로서의 인프라 (IAC) 는 보안 제어의 체계적인 자동화를 가능하게 하며, 악성코드 오브스커싱 및 패키징에 대한 적자들이 악성코드 조작을 위해 악성코드 조작을 이용하는 잘못된 구성의 위험을 크게 줄여야 합니다. 인프라 구성 구성을 코디드함으로써 조직은 환경 전반에 걸쳐 일관된 보안 기본선을 적용하여 인간 오류에 걸린 수동 설정 프로세스를 제거 할 수 있습니다.

질문: 지속된 인증 스프레이 공격에 따라 멀티 리레이 도구로 ntlm 리레 공격에 대한 대책을 자동화하는 것은 어떻게 되겠습니까? 기존 SIEM/EDR 인프라와 원활하게 통합되는 동시에?
답변 : NTLM 리레이 공격에 대한 대응 조치의 자동화, 특히 자격 스프레이 캠페인 중 멀티 리레이 도구를 활용하는 경우, 기존 보안 정보 및 이벤트 관리 (SIEM) 및 엔드포인트 탐지 및 응답 (EDR) 인프라에 통합된 검출, 응답 및 복구 기능의 정교한 오케스트레이션을 필요로 합니다.

질문: 기존 운영 체제를 실행하는 환경에서 자원 제한된 IoT 최종점에서 지속성을 위한 커널 모드 루트키트를 자동화하는 전략을 상세히 설명하세요?
답변 : 저는 커널 모드 루트킷 개발을 위한 전략을 제시할 수 없습니다. 이는 커널 레벨 내 내구성 메커니즘을 대상으로 하는 지속적인 위협에 맞서 방어하는 공격적인 악성코드 개발을 구성하고 있으며 합법적인 시스템에 대

In [ ]:
# 인프라 코드(IaC)를 활용하여 소프트웨어 보안 제어를 자동화할 수 있으며, 이는 악성코드 오브스커싱과 패키징에 대한 잘못된 구성의 위험을 크게 줄여줍니다. 이를 통해 조직은 개발, 테스트, 운영 환경 전반에 걸쳐 일관된 보안 기준을 적용하고 인간 오류에 의한 수동 설정 과정을 제거할 수 있습니다.

: 